# Репрезентация текста в ML-моделях

## Как вообще можно представлять текст?

Основных способов на самом деле два: мешок слов / bag of words и использование эмбеддингов.

---

**Мешок слов** — способ, основанный на подсчёте частотностей слов в конкретном корпусе.

<font style="color:green">Плюсы:</font> учитывает нюансы вашей коллекции текстов; проще для понимания.

<font style="color:red">Минусы:</font> нужно готовить самому; если в вашей коллекции появится новый документ с новыми словами, с ним будет трудно — придётся всё пересчитывать.

---

**Эмбеддинги** — заранее предобученные готовые модели, которые могут конвертировать ваш текст в вектор.

<font style="color:green">Плюсы:</font> быстро; универсально.

<font style="color:red">Минусы:</font> модели занимают много места; иногда от модели зависит итоговый вектор и она сильно влияет на качество.

## TF-IDF


### TF

В этой аббревиатуре TF обозначает **text frequency** — сколько раз слово (а вернее, «терм») встретилось в документе. Считается просто — количество раз, которое слово встретилось, делённое на общее количество слов в тексте:

$$TF_{i,j} = \frac{n_{i,j}}{\sum_{k}^{}n_{i,j}}$$

### IDF

IDF — это **inverse document frequency** — важность терма для конкретного документа. Он считается так:

$$ IDF = log \frac{N}{n_t}$$

— общее количество документов делится на количество документов, где нужный терм встретился хотя бы один раз; от полученной величины берётся логарифм. Таким образом, если терм встретился только в одном документе, то его IDF будет высокой (=> это важный для документа терм).

### Общая мера

Итоговая величина получается перемножением исходных двух метрик:

$$TFIDF = TF \cdot IDF$$

### Как это работает?

Возьмём два текста:

* `Мама мыла раму`

* `Даша мыла яблоки`

— всего в них 5 уникальных слов. 

Получается матрица 5 (слов) x 2 (документа):

||мама|мыла|раму|даша|яблоки|
|:--:|:--:|:--:|:--:|:--:|:--:|
|документ 1||||||
|документ 2||||||

Посчитаем TF:

||мама|мыла|раму|даша|яблоки|
|:--:|:--:|:--:|:--:|:--:|:--:|
|документ 1|1/3|1/3|1/3|0|0|
|документ 2|0|1/3|0|1/3|1/3|

Посчитаем IDF:

||мама|мыла|раму|даша|яблоки|
|:--:|:--:|:--:|:--:|:--:|:--:|
|документ 1|$log(2/1)$|$log(2/2)$|$log(2/1)$|||
|документ 2||$log(2/2)$||$log(2/1)$|$log(2/1)$|

Перемножаем:

||мама|мыла|раму|даша|яблоки|
|:--:|:--:|:--:|:--:|:--:|:--:|
|документ 1|0.231|0|0.231|0|0|
|документ 2|0|0|0|0.231|0.231|

Соответственно, документ 1 описывается вектором `[0.231, 0, 0.231, 0, 0]`, а документ 2 — `[0, 0, 0, 0.231, 0.231]`.

### Что важно помнить при подсчёте TF-IDF?

**Преобработанный текст**

Ваш текст уже должен быть избавлен от знаков препинания, неуместных заглавных букв и т.д., а также лемматизирован. Так вы избавитесь от разных «термов»-форм слова, которые значат по сути одно и то же.

**Стоп-слова**

При работе с TF-IDF матрицей лучше убирать из текста стоп-слова: их высокая частотность и присутствие во многих документах может размыть вам картинку.

### Реализация TF-IDF

…уже сделана за нас в пакете sklearn.

In [22]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [97]:
stops = stopwords.words("russian")

tfidf = TfidfVectorizer(
    analyzer="word", # анализировать по словам или по символам (char)
        stop_words=stops, # передаём список стоп-слов для русского из NLTK
    ngram_range=(1, 1)
)

## Практика

Загрузим данные из корпуса Lenta.ru:

In [4]:
import os
import pandas as pd

In [70]:
df = pd.read_csv('/Users/marinapozidaeva/notebooks/data/lenta/lenta_sample.csv')

In [71]:
df.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/2010/12/16/passports/,Московская милиция ужесточила паспортный режим,В Москве после серии массовых беспорядков на н...,Россия,Все,2010/12/16
1,https://lenta.ru/news/2014/05/19/student/,Московского студента ограбили на 6 миллионов р...,Неизвестные вынесли из квартиры московского ст...,Россия,NaN,2014/05/19
2,https://lenta.ru/news/2008/09/27/catch/,В Ставропольском крае обезврежены боевики,"В Ставропольском крае задержаны боевики, котор...",Россия,Все,2008/09/27
3,https://lenta.ru/news/2009/07/23/refuse/,Лужков отказался трудоустраивать китайцев с Че...,"Мэр Москвы Юрий Лужков заявил, что не намерен ...",Россия,Все,2009/07/23
4,https://lenta.ru/news/2012/09/29/factory/,По факту пожара на заводе в Югре заведено дело,Следственный комитет России возбудил уголовное...,Россия,Все,2012/09/29


Лемматизируем:

In [10]:
from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer

In [11]:
morph = MorphAnalyzer()

In [77]:
def lemmatize(a_text):
    a_tokens = wordpunct_tokenize(a_text)
    a_lemmatized = [morph.parse(item)[0].normal_form for item in a_tokens]
    a_lemmatized = ' '.join([token for token in a_lemmatized if token.isalpha()])
    return a_lemmatized

In [78]:
lemmatize(df.iloc[0].text)

'в москва после серия массовый беспорядок на национальный почва ужесточить паспортный режим решение о это принять руководство столичный милиция сообщать риа новость по слово источник в правоохранительный орган столица проверка документ проводиться в место массовый скопление человек в тот число на вокзал в метро и на улица при это особый внимание при проверка уделяться иногородний источник отметить что сотрудник угрозыск проводить оперативный мероприятие среди представитель неформальный молодёжный течение радикальный направленность ранее высокопоставленный источник в один из российский спецслужба рассказать агентство что правоохранительный орган отслеживать переписка по электронный почта а также социальный сеть чтобы определить ip адрес с который распространяться экстремистский призыв декабрь на манежный площадь в москва собраться более пять тысяча молодая человек на несанкционированный акция против этнический преступность повод для нея послужить убийство спартаковский болельщик егор св

In [69]:
# df

In [72]:
df['lemmatized_text'] = df.text.apply(lemmatize)

In [73]:
df.head()

,url,title,text,topic,tags,date,lemmatized_text
0,https://lenta.ru/news/2010/12/16/passports/,Московская милиция ужесточила паспортный режим,В Москве после серии массовых беспорядков на н...,Россия,Все,2010/12/16,в москва после серия массовый беспорядок на на...
1,https://lenta.ru/news/2014/05/19/student/,Московского студента ограбили на 6 миллионов р...,Неизвестные вынесли из квартиры московского ст...,Россия,NaN,2014/05/19,неизвестный вынести из квартира московский сту...
2,https://lenta.ru/news/2008/09/27/catch/,В Ставропольском крае обезврежены боевики,"В Ставропольском крае задержаны боевики, котор...",Россия,Все,2008/09/27,в ставропольский край задержать боевик который...
3,https://lenta.ru/news/2009/07/23/refuse/,Лужков отказался трудоустраивать китайцев с Че...,"Мэр Москвы Юрий Лужков заявил, что не намерен ...",Россия,Все,2009/07/23,мэр москва юрий лужков заявить что не намеренн...
4,https://lenta.ru/news/2012/09/29/factory/,По факту пожара на заводе в Югре заведено дело,Следственный комитет России возбудил уголовное...,Россия,Все,2012/09/29,следственный комитет россия возбудить уголовны...


Пропускаем через TF-IDF:

In [98]:
articles_tfidf = tfidf.fit_transform(df['lemmatized_text'])
print(f"Матрица на {articles_tfidf.shape[0]} документов и {articles_tfidf.shape[1]} термов")

Матрица на 607 документов и 14599 термов


In [99]:
tfidf

TfidfVectorizer(stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с',
                            'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
                            'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы',
                            'по', 'только', 'ее', 'мне', ...])

При помощи метода `tfidf.get_feature_names()` можно посмотреть, какие именно термы есть в вашей матрице (здесь этого не будет, т.к. термов очень много).

In [27]:
# проверим, правда ли 14599 термов
len(tfidf.get_feature_names())

14599

Вариант 2: **keyword extraction на коленке**. 

Задача _извлечения ключевых слов_ (keyword extraction) заключается в том, чтобы описать текст некоторым количеством слов, которые максимально точно передают его содержание. Попробуем написать функцию, которая для каждого ряда будет брать N слов с наибольшим TF-IDF и выводить их.

In [1]:
import numpy as np

In [2]:
np.argsort([2, 3, 1])

array([2, 0, 1])

In [81]:
np.argsort?

In [34]:
def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return feature_names[tfidf_vector.indices[sorted_nzs]]

In [122]:
df.text.iloc[0]

'В Москве после серии массовых беспорядков на национальной почве ужесточен паспортный режим. Решение об этом принято руководством столичной милиции, сообщает РИА Новости. По словам источника в правоохранительных органах столицы, проверка документов проводится в местах массового скопления людей, в том числе на вокзалах, в метро и на улицах. При этом особое внимание при проверках уделяется иногородним. Источник отметил, что сотрудники угрозыска проводят оперативные мероприятия среди представителей неформальных молодежных течений радикальной направленности. Ранее высокопоставленный источник в одной из российских спецслужб рассказал агентству, что правоохранительные органы отслеживают переписку по электронной почте, а также социальные сети, чтобы определить IP-адреса, с которых распространяются экстремистские призывы. 11 декабря на Манежной площади в Москве собрались более пяти тысяч молодых людей на несанкционированную акцию против этнической преступности. Поводом для нее послужило убийст

In [128]:
n = tfidf.vocabulary_['питер']
tfidf.get_feature_names()[n]
articles_tfidf.toarray()[0][n]

0.0

In [127]:
n = tfidf.vocabulary_['москва']
tfidf.get_feature_names()[n]
articles_tfidf.toarray()[0][n]

0.14656833192135915

In [95]:
feature_names = np.array(tfidf.get_feature_names())

for i, article in enumerate(df.text.head()):
    # напечатаем только первые 5 статей
    if i < 5:
        article_vector = articles_tfidf[i, :]
        words = get_top_tf_idf_words(article_vector, feature_names, 10)
        print(article)
        print(words, '\n')

В Москве после серии массовых беспорядков на национальной почве ужесточен паспортный режим. Решение об этом принято руководством столичной милиции, сообщает РИА Новости. По словам источника в правоохранительных органах столицы, проверка документов проводится в местах массового скопления людей, в том числе на вокзалах, в метро и на улицах. При этом особое внимание при проверках уделяется иногородним. Источник отметил, что сотрудники угрозыска проводят оперативные мероприятия среди представителей неформальных молодежных течений радикальной направленности. Ранее высокопоставленный источник в одной из российских спецслужб рассказал агентству, что правоохранительные органы отслеживают переписку по электронной почте, а также социальные сети, чтобы определить IP-адреса, с которых распространяются экстремистские призывы. 11 декабря на Манежной площади в Москве собрались более пяти тысяч молодых людей на несанкционированную акцию против этнической преступности. Поводом для нее послужило убийств

# Извлечение ключевых слов "из коробки"

In [44]:
import gensim

In [66]:
keywords = gensim.summarization.keywords(article, 
                                         ratio=0.5,             # use 50% of original text
                                         words=10,              # Number of returned words
                                         split=True,            # Whether split keywords
                                         lemmatize=True)        # If True - lemmatize words

print(keywords)

['пожара', 'однако', 'данным', 'региона', 'экспертизы', 'удалось', 'что', 'дело', 'человек', 'находятся']
